# Palo Alto Analysis

## Data sources

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import scdata as sc
from scdata._config import config
config._out_level = 'DEBUG'

In [ ]:
test = sc.Test('PALO_ALTO')

### Test create

In [ ]:
test.add_device(sc.Device(blueprint='sck_21_co2', descriptor={'source': 'api', 'id': 14129, 'frequency': '1Min'}))

In [ ]:
test.add_device(sc.Device(blueprint='sck_21_co2', descriptor={'source': 'api', 'id': 14133, 'frequency': '1Min'}))

In [ ]:
test.add_devices_list(blueprint = 'csic_station', devices_list = ['43171002', '43148003', '08019043', '08019044', '08019004'])

In [ ]:
test.create(force = True)

### Load

In [ ]:
test.load(options = {'min_date': '2021-05-28'})

In [ ]:
print ("did = '43171002' Vila-seca (Suburban background)")
print ("did = '43148003' Tarragona (Suburban industrial)")
print ("did = '08019043' Barcelona Eixample (Urban Traffic)")
print ("did = '08019044' Barcelona Gracia (Urban Traffic)")
print ("did = '08019004' Barcelona Poblenou (Urban Background)")

In [ ]:
metric = {f'SCD30_CO2_COR': {'process': 'poly_ts',
                           'kwargs': {'channels': ['SCD30_CO2'],
                                      'coefficients': [1],
                                      'extra_term': 100}
                        }}

test.devices['14133'].add_metric(metric)

In [ ]:
test.devices['14133'].process()

### DadesObertes Sources

In [ ]:
from scdata.io.device_api import DadesObertesApiDevice

In [ ]:
# Barcelona stations
stations = DadesObertesApiDevice.get_world_map(full = True)
stations

In [ ]:
list(set(list(stations['contaminant'])))

In [ ]:
stations[((stations['contaminant'] == 'PM1') | (stations['contaminant'] == 'PM2.5'))]

In [ ]:
stations[(stations['municipi'] == 'Barcelona') & ((stations['contaminant'] == 'PM10') | (stations['contaminant'] == 'CO2'))]

In [ ]:
stations[(stations['contaminant'] == 'PM2.5') | (stations['contaminant'] == 'PM1') | (stations['contaminant'] == 'CO2')]

In [ ]:
print (list(set(list(stations[(stations['contaminant'] == 'PM2.5') | (stations['contaminant'] == 'PM1') | (stations['contaminant'] == 'CO2')].index))))
print (list(set(list(stations[(stations['contaminant'] == 'PM2.5') | (stations['contaminant'] == 'PM1') | (stations['contaminant'] == 'CO2')].contaminant))))
print (list(set(list(stations[(stations['contaminant'] == 'PM2.5') | (stations['contaminant'] == 'PM1') | (stations['contaminant'] == 'CO2')].municipi))))
print (list(set(list(stations[(stations['contaminant'] == 'PM2.5') | (stations['contaminant'] == 'PM1') | (stations['contaminant'] == 'CO2')].tipus_estacio))))

In [ ]:
devicetr = DadesObertesApiDevice(43148003)

In [ ]:
devicetr.get_device_sensors()

### TS CO2 Both

In [ ]:
traces = {
            "1": {"devices": ["14129"],
                  "channel": ["SCD30_CO2"],
                  "subplot": 1},
            "2": {"devices": ["14133"],
                  "channel": ["SCD30_CO2_COR"],
                  "subplot": 1}    
        }
options = {'frequency': '10Min'}
formatting = {'width': 1200, 'title': 'CO2', 'ylabel': {1: 'CO2 (ppm)'}}
fco2ts = test.ts_uplot(traces=traces, options = options, formatting = formatting)
fco2ts

### TS Noise Both

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["NOISE_A"],
                  "subplot": 1}
        }
options = {'frequency': '10Min'}
formatting = {'width': 1200, 'title': 'Noise (dBA)', 'ylabel': {1: 'Noise (dBA)'}, 'style': 'grayscale'}
fnoisezoom = test.ts_uplot(traces=traces, options = options, formatting = formatting);
fnoisezoom

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["BATT"],
                  "subplot": 1}
        }
options = {'frequency': '5Min'}
formatting = {'width': 1000}
test.ts_uplot(traces=traces, options = options, formatting = formatting)

### TS TEMP Both

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["TEMP"],
                  "subplot": 1}
        }
options = {'frequency': '5Min'}
formatting = {'width': 1200, 'title': 'Temperatura', 'ylabel': {1: 'Temperatura (ºC)'}}
ftempts = test.ts_uplot(traces=traces, options = options, formatting = formatting)
ftempts

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["HUM"],
                  "subplot": 1}
        }
options = {'frequency': '5Min'}
formatting = {'width': 1200, 'title': 'Humedad', 'ylabel': {1: 'Humedad relativa (%rh)'}}
fhumts = test.ts_uplot(traces=traces, options = options, formatting = formatting)
fhumts

#### Temperature Max (14133)

In [ ]:
from scdata.test.plot.plot_tools import prepare_data, groupby_session

traces = {
            "1": {"devices": "14133",
                  "channel": "TEMP"} 
        }

options = {"frequency": '10Min', "clean_na": None}

formatting = {"frequency_hours": 24, 'session': '1D'
              }

# Get dataframe
df, _ = prepare_data(test, traces, options)
gskwags = {'frequency_hours': formatting['frequency_hours']}

dfgb, _, _, _ = groupby_session(df, **gskwags)

df = dfgb.pivot(columns='session').resample('1D').max()
df = df.resample('2D').mean()
df.index = df.index.strftime('%Y-%m-%d')
figtempmax = df.plot.bar(figsize=(20,8), fontsize=14);
figtempmax.set_ylabel('Temperatura (ºC)', fontsize=16);

figtempmax.set_xlabel(None);
figtempmax.figure.suptitle('Temperatura (ºC) - 24h max', fontsize = 14);
figtempmax.legend(['Temperatura - XYZ']);
figtempmax.figure.set_facecolor('white')
figtempmax

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["PRESS"],
                  "subplot": 1}
        }
options = {'frequency': '5Min'}
formatting = {'width': 1000}
test.ts_uplot(traces=traces, options = options, formatting = formatting)

### TS PM1 Both

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["PM_1"],
                  "subplot": 1}
        }
options = {'frequency': '1H'}
formatting = {'width': 1200, 'title': 'PM1 (ug/m3)', 'ylabel': {1: 'PM1 (ug/m3)'}, 'style': 'grayscale'}
fpm1ts = test.ts_uplot(traces=traces, options = options, formatting = formatting);
fpm1ts

### TS PM2.5 Both

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["PM_25"],
                  "subplot": 1}
        }
options = {'frequency': '1H'}
formatting = {'width': 1200, 'title': 'PM25 (ug/m3)', 'ylabel': {1: 'PM25 (ug/m3)'}, 'style': 'grayscale'}
fpm25zoom = test.ts_uplot(traces=traces, options = options, formatting = formatting);
fpm25zoom

### TS PM10 Both

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["PM_10"],
                  "subplot": 1}
        }
options = {'frequency': '1H'}
formatting = {'width': 1200, 'title': 'PM10 (ug/m3)', 'ylabel': {1: 'PM10 (ug/m3)'}, 'style': 'grayscale'}
fpm10ts = test.ts_uplot(traces=traces, options = options, formatting = formatting);
fpm10ts

### Noise 14133 (XYZ)

#### Daily distribution

In [ ]:
traces = {
            "1": {"devices": "14133",
                  "channel": "NOISE_A"} 
        }

options = {
            "show": True,
            "frequency": '1Min',
            "clean_na": None,
            #"max_date": '2020-11-20',
            #"min_date": '2020-02-08'
            }

formatting = {"title": "Distribución diaria de ruido",
              "grid": True,
              "legend": True,
              "fontsize": 14,
              "title_fontsize": 16,
              "height": 10,
              "suptitle_factor": 0.92,
              "ylabel": "Noise (dBA)"
              }

fnoiseday = test.box_plot(traces = traces, options = options, formatting = formatting);

#### In group per 6h

In [ ]:
from scdata.test.plot.plot_tools import prepare_data, groupby_session

traces = {
            "1": {"devices": "14133",
                  "channel": "NOISE_A"} 
        }

options = {"clean_na": None,"frequency": '1Min'}

formatting = {"frequency_hours": 6, 'session': '1D'
              }

# Get dataframe
df, _ = prepare_data(test, traces, options)
gskwags = {'frequency_hours': formatting['frequency_hours']}

dfgb, _, _, _ = groupby_session(df, **gskwags)

In [ ]:
dfgb.pivot(columns='session').resample(formatting['session']).mean().mean()

#### Events analysis

In [ ]:
traces = {
            "1": {"devices": "14133",
                  "channel": "NOISE_A"} 
        }

options = {
            "show": True,
            "frequency": '1Min',
            "clean_na": None,
            #"max_date": '2020-11-20',
            #"min_date": '2020-02-08'
            }

formatting = {"title": "Distribución Ruido ",
              "grid": True,
              "legend": True,
              "height": 10,
              "suptitle_factor": 0.92,
              "periods": {"dates": ['2021-06-22', '2021-06-25', '2021-06-28'],
                          "labels": ["San Juan", "Normal days"]
                         },
              }

fnoisesj = test.box_plot(traces = traces, options = options, formatting = formatting);

In [ ]:
traces = {
            "1": {"devices": "14129",
                  "channel": "NOISE_A"} 
        }

options = {
            "show": True,
            "frequency": '1Min',
            "clean_na": None,
            #"max_date": '2020-11-20',
            #"min_date": '2020-02-08'
            }

formatting = {"title": "Distribución ruido - Eventos",
              "grid": True,
              "legend": True,
              "height": 10,
              "suptitle_factor": 0.92,
              "periods": {"dates": ['2021-06-23', '2021-06-24', None],
                          "labels": ["San Juan", "Normal days"]
                         },
              }

fnoisesj = test.box_plot(traces = traces, options = options, formatting = formatting);

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["NOISE_A"],
                  "subplot": 1}
        }
options = {'frequency': '30Min', 'min_date': '2021-06-22', 'max_date': '2021-06-26', 'show': False}
formatting = {'width': 18, 'title': 'Rudio - Análisis en Festividad de San Juan', 'ylabel': {1: 'Ruido (dBA)'}, 'style': 'grayscale', 'fontsize': 14, 'yrange': {1: [25, 100]}}
fnoisesj = test.ts_plot(traces=traces, options = options, formatting = formatting);
fnoisesj.set_facecolor('white');

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["NOISE_A"],
                  "subplot": 1}
        }
options = {'frequency': '30Min', 'min_date': '2021-06-29', 'max_date': '2021-07-09', 'show': False}
formatting = {'width': 18, 'title': 'Rudio - Análisis en otros eventos', 'ylabel': {1: 'Ruido (dBA)'}, 'style': 'grayscale', 'fontsize': 14, 'yrange': {1: [25, 100]}}
fnoiseevent = test.ts_plot(traces=traces, options = options, formatting = formatting);
fnoiseevent.set_facecolor('white');

In [ ]:
from scdata.test.plot.plot_tools import prepare_data, groupby_session

traces = {
            "1": {"devices": "14133",
                  "channel": "NOISE_A"} 
        }

options = {"frequency": '10Min', "clean_na": None}

formatting = {"frequency_hours": 24, 'session': '1D'
              }

# Get dataframe
df, _ = prepare_data(test, traces, options)
gskwags = {'frequency_hours': formatting['frequency_hours']}

dfgb, _, _, _ = groupby_session(df, **gskwags)

df = dfgb.pivot(columns='session').resample('1D').max()
df = df.resample('2D').mean()
df.index = df.index.strftime('%Y-%m-%d')
fignoisemax = df.plot.bar(figsize=(20,8), fontsize=14);
fignoisemax.set_ylabel('Ruido (dBA)', fontsize=16);

fignoisemax.set_xlabel(None);
fignoisemax.figure.suptitle('Ruido (dBA) - 24h max', fontsize = 14);
fignoisemax.legend(['Ruido - XYZ']);
fignoisemax.figure.set_facecolor('white')
fignoisemax

### PM2.5 14129 (entrada)

#### Daily distribution

In [ ]:
traces = {
            "1": {"devices": "14129",
                  "channel": "PM_25"} 
        }

options = {
            "show": True,
            "frequency": '1Min',
            "clean_na": None,
            #"max_date": '2020-11-20',
            #"min_date": '2020-02-08'
            }

formatting = {"title": "PM1 Distribution",
              "grid": True,
              "legend": True,
              "height": 10,
              "suptitle_factor": 0.92
              }

fpmday = test.box_plot(traces = traces, options = options, formatting = formatting);

#### Events analysis

In [ ]:
traces = {
            "1": {"devices": "all",
                  "channel": ["PM_25"],
                  "subplot": 1}
        }
options = {'frequency': '1H', 'min_date': '2021-06-22', 'max_date': '2021-06-26', 'show': False}
formatting = {'width': 18, 'title': 'PM25 (ug/m3) - Análisis en Festividad de San Juan', 'ylabel': {1: 'PM25 (ug/m3)'}, 'style': 'grayscale', 'fontsize': 14}
fpm25sj = test.ts_plot(traces=traces, options = options, formatting = formatting);
fpm25sj.set_facecolor('white');

In [ ]:
traces = {
            "1": {"devices": "14129",
                  "channel": "PM_25"} 
        }

options = {
            "show": True,
            "frequency": '1Min',
            "clean_na": None,
            #"max_date": '2020-11-20',
            #"min_date": '2020-02-08'
            }

formatting = {"title": "PM1 Distribution",
              "grid": True,
              "legend": True,
              "height": 10,
              "suptitle_factor": 0.92,
              "periods": {"dates": ['2021-06-23', '2021-06-24', None],
                          "labels": ["San Juan", "Normal days"]
                         },
              }

fpmsj = test.box_plot(traces = traces, options = options, formatting = formatting);

#### In group per day

In [ ]:
from scdata.test.plot.plot_tools import prepare_data, groupby_session

traces = {
            "1": {"devices": "14129",
                  "channel": "PM_25"} 
        }

options = {"frequency": '1Min', "clean_na": None}

formatting = {"frequency_hours": 24, 'session': '1D'
              }

# Get dataframe
df, _ = prepare_data(test, traces, options)
gskwags = {'frequency_hours': formatting['frequency_hours']}

dfgb, _, _, _ = groupby_session(df, **gskwags)

In [ ]:
df = dfgb.pivot(columns='session').resample('1D').mean().loc['2021-06-22':'2021-06-28', :]['PM_25_14129']
df.index = df.index.strftime('%Y-%m-%d')
figpm25avgsj = df.plot.bar(figsize=(20,8), fontsize=14);
figpm25avgsj.set_ylabel('PM 2.5 (ug/m3)', fontsize=16);
figpm25avgsj.set_ylim([0, 50])
figpm25avgsj.set_xlabel(None);
figpm25avgsj.figure.suptitle('PM 2.5 (ug/m3) - promedio de 24h durante festividad de San Juan', fontsize = 16);
figpm25avgsj.legend(['PM 2.5 - XYZ'], frameon=True);
figpm25avgsj.figure.set_facecolor('white')
figpm25avgsj

In [ ]:
traces = {
            "1": {"devices": ["14129", "14133"],
                  "channel": ["PM_25"],
                  "subplot": 1}
        }
options = {'frequency': '1D', 'show': False}
formatting = {'width': 18, 'title': 'PM25 (ug/m3)', 'ylabel': {1: 'PM25 (ug/m3)'}, 'style': 'grayscale', 'fontsize': 14, 'yrange': {1: [0, 90]}}
figpm25avg24 = test.ts_plot(traces=traces, options = options, formatting = formatting);
figpm25avg24.set_facecolor('white');

### CO2 14129 (entrada)

#### Daily distribution

In [ ]:
traces = {
            "1": {"devices": "14129",
                  "channel": "SCD30_CO2"} 
        }

options = {
            "show": True,
            "frequency": '1Min',
            "clean_na": None,
            #"max_date": '2020-11-20',
            #"min_date": '2020-02-08'
            }

formatting = {"title": "Distribución diaria de CO2 (exterior)",
              "grid": True,
              "legend": True,
              "fontsize": 14,
              "title_fontsize": 16,
              "height": 10,
              "suptitle_factor": 0.92,
              "ylabel": "Noise (dBA)"
              }

fco2day = test.box_plot(traces = traces, options = options, formatting = formatting);

#### Events analysis

In [ ]:
traces = {
            "1": {"devices": ["14129"],
                  "channel": ["SCD30_CO2"],
                  "subplot": 1},
            "2": {"devices": ["14133"],
                  "channel": ["SCD30_CO2_COR"],
                  "subplot": 1
                 }
        }
options = {'frequency': '30Min', 'min_date': '2021-06-22', 'max_date': '2021-06-26', 'show': False}
formatting = {'width': 18, 'title': 'CO2 - Análisis en Festividad de San Juan', 'ylabel': {1: 'CO2 (ppm)'}, 'style': 'grayscale', 'fontsize': 14}
fco2sj = test.ts_plot(traces=traces, options = options, formatting = formatting);
fco2sj.set_facecolor('white');

In [ ]:
traces = {
            "1": {"devices": ["14129"],
                  "channel": ["SCD30_CO2"],
                  "subplot": 1},
            "2": {"devices": ["14133"],
                  "channel": ["SCD30_CO2_COR"],
                  "subplot": 1
                 }
        }
options = {'frequency': '30Min', 'min_date': '2021-06-29', 'max_date': '2021-07-09', 'show': False}
formatting = {'width': 18, 'title': 'CO2 - Análisis en otros eventos', 'ylabel': {1: 'CO2 (ppm)'}, 'style': 'grayscale', 'fontsize': 14}
fco2event = test.ts_plot(traces=traces, options = options, formatting = formatting);
fco2event.set_facecolor('white');

## Metrics

### PM10 AVG

In [ ]:
for device in test.devices:
    print (f"{device} PM10 avg: {test.devices[device].readings['PM_10'].mean()}")

### PM2.5 AVG

In [ ]:
for device in test.devices:
    if 'PM_25' not in test.devices[device].readings: continue
    print (f"{device} PM25 avg: {test.devices[device].readings['PM_25'].mean()}")

### PM1 AVG

In [ ]:
for device in test.devices:
    if 'PM_1' not in test.devices[device].readings: continue
    print (f"{device} PM1 avg: {test.devices[device].readings['PM_1'].mean()}")

### CO2 AVG

In [ ]:
for device in test.devices:
    if 'SCD30_CO2' not in test.devices[device].readings: continue
    print (f"{device} SCD30_CO2 avg: {test.devices[device].readings['SCD30_CO2'].mean()}")
    if 'SCD30_CO2_COR' not in test.devices[device].readings: continue
    print (f"{device} SCD30_CO2_COR avg: {test.devices[device].readings['SCD30_CO2_COR'].mean()}")    

### NOISE AVG

In [ ]:
for device in test.devices:
    if 'NOISE_A' not in test.devices[device].readings: continue
    print (f"{device} NOISE avg: {test.devices[device].readings['NOISE_A'].mean()}")

In [ ]:
for device in test.devices:
    if 'NOISE_A' not in test.devices[device].readings: continue
    print (f"{device} NOISE std: {test.devices[device].readings['NOISE_A'].std()}")

## Report

In [ ]:
test.descriptor['author'] = 'Óscar González'
test.descriptor['project'] = 'Fundación Palo Alto'
test.descriptor['date'] = 'De 28-05-2021 a 01-10-2021'
test.descriptor['type_test'] = 'Exterior - Alimentación a Panel Solar'
test.descriptor['comment'] = 'Primer análisis de cuatro meses (verano 2021)'

In [ ]:
test.content = {}

In [ ]:
text = '''
 Este análisis realiza la comparativa del periodo de medidas desde el 28/05/2021 hasta el 01/10/2021 como primer tramo
 de dos unidades de medición de calidad de aire en el recinto de Palo Alto. Las unidades cuentan con medidores de PM (Partículas en suspensión), 
 CO2, ruido, temperatura, humedad y presión barométrica. El primero de ellos se sitúa en la entrada del recinto (14129), 
 mientras que el segundo se encuentra en el lateral de la Nave XYZ (14133). Ambos sensores se encuentran a una altura de aproximadamente 
 3m y están alimentados mediante panel solar.
 <br>
 <br>
 Adicionalmente, se añaden al análisis estaciones de la Agencia de Salud Pública y la Generalitat de Cataluña, situadas en zonas de 
 ciudad tales como el Eixample (estación urbana de tráfico), Gracia (estación urbana de tráfico), Poblenou (estación urbana de fondo), 
 Vila-Seca (estación suburbana de fondo) y Tarragona (estación suburbana industrial).
 <br>
 <br>
 La estaciones de Vila-Seca y Tarragona han sido seleccionadas únicamente como referencia para las 
 comparaciones de las medidas de PM1 y PM2.5, ya que son las únicas comparables con las medidas de referencia
 de los Smart Citizen Kits instalados en Palo Alto. Ninguna estación de referencia cuenta con
 medidas de CO2 o ruido en exterior, por lo que esta comparativa se realiza a través de datos tabulados. Las medidas
 de PM10 no son utilizadas como referencia a pesar de encontrarse en todas las estaciones ya que el sensor
 de PM10 en el Smart Citizen Kit no presenta buena eficiencia en las medidas de este tamaño de partículas,
 por lo que no son comparables a las medidas oficiales.
 <br>
 <br>
 <strong>Importante</strong> notar que los sensores de PM del Smart Citizen Kit no utilizan el 
 mismo principio de funcionamiento que los usados en los instrumentos de referencia, por lo que el 
 análisis aquí realizado únicamente debe servir como referencia sin validez científica.
'''

test.add_content(title = 'Introducción', figure = None, text = text)

In [ ]:
text = '''
 Los datos de ruido se encuentran representados en el gráfico inferior. Es posible <em>hacer zoom</em> en distintas partes del mismo
 para analizar más en detalle. 
 <br>
 <br>
 El nivel de ruido medio de cada medidor es de:
 <br>
 <ul>
 <li><strong>14129 (entrada)</strong>: 42.2 (escala dBA)</li>
 <li><strong>14133 (XYZ)</strong>: 40.8 (escala dBA)</li>
 </ul>
 <br>
 Estos niveles de ruido, en nivel promedio, son similares a aquellos que se pueden encontrar en un espacio de oficina tranquilo o una biblioteca
 (<a href="https://noiseawareness.org/info-center/common-noise-levels/">ref</a>). 
'''

test.add_content(title = 'Análisis Datos de Ruido', iframe = fnoisezoom, text = text, show_title = True, force = True)

In [ ]:
text = '''
 En el gráfico inferior se muestran datos de dispersión y comparativa día vs. noche en los datos de ruido del espacio para el sensor de la XYZ como ejemplo,
 aunque los datos son comparables en cada sensor. Debajo se muestran promedios de ruido en diferentes horarios:
 <ul>
 <li><strong>0am-6am</strong>: 33.0 (escala dBA)
 <li><strong>6am-12am</strong>: 41.1 (escala dBA)
 <li><strong>12pm-18pm</strong>: 46.5 (escala dBA)
 <li><strong>18pm-0am</strong>: 42.4 (escala dBA)
 </ul>
 Estos datos muestran una mayor actividad en la tarde, con valores más cercanos a los 50 dBA de media, aunque normalmente afectados en promedio por la presencia
 de eventos que ocurren en la tarde. Los valores nocturnos por el contrario son inferiores a 33 dBA, mostrando que la actividad de fondo del recinto es 
 silenciosa, que denota ausencia de maquinaria, tráfico o similares.

'''

test.add_content(title = 'Datos intradiarios', figure = fnoiseday, text = text, show_title = False, force = True)

In [ ]:
text = '''
    Debajo se muestra el en ruido de los eventos (San Juan):
'''
test.add_content(title = 'Eventos - efecto ruido', figure = fnoisesj, text = text, show_title = False, force = True)

In [ ]:
text = '''
    Y en la figura inferior se muestran datos de ruido centrados en otros eventos, donde los máximos a los 90dBA:
'''
test.add_content(title = 'Eventos - efecto ruido (2)', figure = fnoiseevent, text = text, show_title = False, force = True)

In [ ]:
text = '''
    Debajo se muestra el ruido máximo promediado diario, con un valor valle cercano a los 50dBA, aunque superando en repetidas ocasiones valores de 80 dBA:
'''
test.add_content(title = 'Eventos - efecto ruido máximo', figure = fignoisemax.figure, text = text, show_title = False, force = True)

In [ ]:
text = '''
 Los datos de PM2.5 (partículas en suspensión de tamaño menor a 2.5um o <em>fine particulate matter</em>) se encuentran representados en el gráfico inferior. Es posible _hacer zoom_ en distintas partes del mismo
 para analizar más en detalle. El nivel de PM2.5 medio de cada medidor es de:
 <br>
 <ul>
 <li><strong>14129 (entrada)</strong>: 6.3 (ug/m3)</li>
 <li><strong>14133 (XYZ)</strong>: 6.4 (ug/m3)</li>
 </ul>
 En comparación con los datos de un periodo similar, los datos son inferiores a aquellos encontrados incluso en estaciones de referencia en entornos <em>suburbanos</em>:
 <br>
 <ul>
 <li><strong>43148003 (Tarragona - Estación suburbana industrial)</strong>: 10.2 (ug/m3)</li>
 <li><strong>43171002 (Vila-seca - Estación suburbana de fondo)</strong>: 10.6 (ug/m3)</li>
 </ul>
 Estos niveles de PM25, en nivel promedio, son inferiores a aquellos que se pueden encontrar en zonas <em>suburbanas</em> de carácter rural, y aunque no haya indicadores 
 de estos valores en zonas rurales o similares a un parque natural, los indicadores de la OMS (<a href="https://apps.who.int/iris/handle/10665/345329">ref</a>) se satisfacen
 en valor promedio en Palo Alto, <strong>estando su valor por debajo de 10ug/m3 como valor recomendado</strong>.
'''

test.add_content(title = 'Análisis Datos de PM2.5', iframe = fpm25zoom, text = text, show_title = True, force = True)

In [ ]:
text = '''
    Adicionalmente, la media diaria nunca excede los valores máximos recomendados (25ug/m3 en el caso más restrictivo) como mostrado en la figura inferior:
'''

test.add_content(title = 'Análisis Datos de PM2.5 (2)', figure = figpm25avg24, text = text, show_title = False, force = True)

In [ ]:
text = '''
    Debajo se muestra el en PM2.5 de los eventos (San Juan):
'''
test.add_content(title = 'Eventos - efecto en PM2.5', figure = fpm25sj, text = text, show_title = False, force = True)

In [ ]:
text = '''
    Debajo se muestra el efecto promedio en PM2.5 por día de un evento como la Fiesta de San Juan:
'''
test.add_content(title = 'Eventos - efecto en PM2.5 (2)', figure = figpm25avgsj.figure, text = text, show_title = False, force = True)

In [ ]:
text = '''
 Los datos de PM10 (partículas en suspensión de tamaño menor a 10um o <em>particulate matter</em>) se encuentran representados en el gráfico inferior. Es posible _hacer zoom_ en distintas partes del mismo
 para analizar más en detalle. <strong>Importante</strong>: los valores de PM10 medidos por el Smart Citizen Kit no son enteramente comparables a los
 medidos por las estaciones de referencia, ya que el sensor utilizado es conocido por "subestimar" las medidas. Igualmente aquí abajo se indican los valores 
 promedios como referencia. El nivel de PM10 medio de cada medidor es de:
 <br>
 <ul>
 <li><strong>14129 (entrada)</strong>: 6.9 (ug/m3)</li>
 <li><strong>14133 (XYZ)</strong>: 7.5 (ug/m3)</li>
 </ul>
 En comparación con los datos de un periodo similar, los datos son inferiores a aquellos 
 <br>
 <ul>
 <li><strong>43148003 (Tarragona - Estación suburbana industrial)</strong>: 23.3 (ug/m3)</li>
 <li><strong>43171002 (Vila-seca - Estación suburbana de fondo)</strong>: 18.8 (ug/m3)</li> 
 <li><strong>08019004 (Barcelona Poblenou - Estación urbana de fondo)</strong>: 32.0 (ug/m3)</li>
 <li><strong>08019043 (Barcelona Example - Estación urbana de tráfico)</strong>: 27.9 (ug/m3)</li>
 <li><strong>08019044 (Barcelona Gracia - Estación urbana de tráfico)</strong>: 23.0 (ug/m3)</li>
 </ul>
'''

test.add_content(title = 'Análisis Datos de PM10', iframe = fpm10ts, text = text, show_title = True, force = True)

In [ ]:
text = '''
 Los datos de PM1 (partículas en suspensión de tamaño menor a 1um) se encuentran representados en el gráfico inferior. Es posible <em>hacer zoom</em> en distintas partes del mismo
 para analizar más en detalle. El nivel de PM1 medio de cada medidor es de:
 <br>
 <ul>
 <li><strong>14129 (entrada)</strong>: 5.6 (ug/m3)</li>
 <li><strong>14133 (XYZ)</strong>: 5.3 (ug/m3)</li>
 </ul>
 En comparación con los datos de un periodo similar, los datos son comparables 
 <br>
 <ul>
 <li><strong>43148003 (Tarragona - Estación suburbana industrial)</strong>: 6.00263435194942</li>
 <li><strong>43171002 (Vila-seca - Estación suburbana de fondo)</strong>: 7.28
 </ul>
 Estos niveles de PM1, en nivel promedio, son similares a aquellos que se pueden encontrar en un ambiente fuera de la ciudad, en zonas suburbanas. 
 Es importante notar que estos valores no tienen un valor <em>regulado</em> por normativas o recomendaciones como las de la OMS, y es por este motivo que no
 se presentan datos referenciados al respecto.
'''

test.add_content(title = 'Análisis Datos de PM1', iframe = fpm1ts, text = text, show_title = True, force = True)

In [ ]:
text = '''
 Los datos de CO2 (ppm) se encuentran representados en el gráfico inferior. Es posible <em>hacer zoom</em> en distintas partes del mismo
 para analizar más en detalle. El nivel de CO2 medio de cada medidor es de:
 <br>
 <ul> 
 <li><strong>14129 (entrada)</strong>: 421.8 (ppm)</li>
 <li><strong>14133 (XYZ)</strong>: 432.4 (ug/m3)</li>
 </ul>
 Estos niveles de CO2, en nivel promedio, son razonablemente similares a valore de fondo (entorno 400-450ppm) en era post-industrial.
'''

test.add_content(title = 'Análisis Datos de CO2', iframe = fco2ts, text = text, show_title = True, force = True)

In [ ]:
text = '''
    Estos valores de CO2 se superan en eventos, sobre todo en el sensor de la entrada del recinto. Sin embargo, estos valores, 
    al estar en exterior, no suponen un efecto adverso en absoluto, ya que los valores normales en un edificio ocupado se encuentran entre 600-800ppm.
'''

test.add_content(title = 'Análisis Datos de CO2 (eventos)', figure = fco2event, text = text, show_title = False, force = True)

In [ ]:
text = '''
 Los datos de temperatura y humedad se encuentran representados abajo. No es posible hacer una comparación de estos datos con otros valores de referencia,
 al ser medidas puntuales. 
'''

test.add_content(title = 'Análisis Datos de Temperatura y Humedad', iframe = ftempts, text = text, show_title = True, force = True)

In [ ]:

test.add_content(title = 'Análisis Datos de Temperatura y Humedad (2)', iframe = fhumts, text = None, show_title = False, force = True)

In [ ]:
text = 'Sin embargo, se muestra debajo una medida media diaria de temperatura como referencia.'
test.add_content(title = 'Análisis Datos de Temperatura y Humedad (3)', figure = figtempmax.figure, text = text, show_title = False, force = True)

## Save

In [ ]:
report_path, _ = test.to_html(title='Palo Alto Analysis', devices_summary=True);

In [ ]:
!open $report_path

## Upload

In [ ]:
s3_report_path = f'reports/{test.full_name}_1.html'
url = s3handler.upload(report_path, s3_report_path)